<a href="https://colab.research.google.com/github/kim-jason/PowerliftingDataset/blob/main/Powerlifting_Data_Science_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Powerlifting Dataset EDA

Name: Jason Kim  <br>
Email: jason.s.kim20@gmail.com <br>
Linkedin: https://www.linkedin.com/in/jasonks/

## Introduction

### Overview of notebook


The purpose of this notebook is to analyze a dataset consisting of powerlifting meets in order to identify trends and insights by age, country, federation, and other provided and generated features. <br>

We will be using the CSV dataset from [OpenPowerlifting](https://openpowerlifting.gitlab.io/opl-csv/introduction.html), an open source site that regularly compiles official results from powerlifting competitions around the world. <br>

### Overview of powerlifting

Powerlifting is a sport consisting of three specific lifts: squat, bench press, and deadlift. The sum of the best lift in each discipline determines the winner in a competition. Lifters compete in given age groups and weight categories. 

 <!-- [![Everything Is AWESOME](http://i.imgur.com/Ot5DWAW.png)](https://youtu.be/StTqXEQ2l-Y?t=35s "Everything Is AWESOME") -->

 TO DO: Add description fo three


## Data import

In [1]:
# Import libraries used in the notebook
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Download the raw CSV dataset (note I saved this CSV file to my personal Google Drive as it makes it faster to download and use in Google Colab)
! gdown --id 1b15_F2lH2sHI02m7F1HGQkDw3Bl25-X9

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1b15_F2lH2sHI02m7F1HGQkDw3Bl25-X9
To: /content/openpowerlifting-2022-08-06-98216f16.csv
100% 540M/540M [00:02<00:00, 219MB/s]


In [3]:
# Create dataframe with the raw CSV dataset
df = pd.read_csv('openpowerlifting-2022-08-06-98216f16.csv')
df.head()
print(len(df))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (33,35,38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2755243


In [4]:
# Describe the contents of the dataframe
df.describe().apply(lambda s: s.apply('{0:.1f}'.format))

,Age,BodyweightKg,Squat1Kg,Squat2Kg,Squat3Kg,Squat4Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,...,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Deadlift4Kg,Best3DeadliftKg,TotalKg,Dots,Wilks,Glossbrenner,Goodlift
count,1720150.0,2719385.0,710550.0,702587.0,682372.0,7313.0,1821444.0,1158272.0,1144372.0,1107394.0,...,810608.0,795214.0,759616.0,19399.0,1970884.0,2564651.0,2541034.0,2541034.0,2541034.0,2318067.0
mean,30.8,84.1,113.5,95.8,36.6,75.7,174.3,83.5,56.3,-16.5,...,159.6,131.8,20.7,80.1,189.1,375.4,273.0,272.0,256.5,63.9
std,13.2,22.7,140.2,164.7,193.0,182.8,69.0,101.3,126.2,141.0,...,108.3,157.4,211.4,187.3,62.2,206.0,129.6,129.1,122.7,16.3
min,0.0,10.0,-555.0,-580.0,-600.5,-550.0,-508.0,-515.0,-589.7,-575.0,...,-461.0,-502.5,-587.5,-500.0,-410.0,1.0,0.7,0.7,0.6,0.5
25%,20.5,67.1,90.0,75.0,-155.0,-88.0,122.5,55.0,-45.0,-135.0,...,125.0,115.0,-202.5,-75.5,140.0,200.0,141.6,141.0,133.5,52.2
50%,27.5,81.9,145.0,145.0,115.0,131.0,170.0,105.0,95.0,-60.0,...,180.0,177.5,120.0,142.9,188.2,351.5,294.6,293.6,274.1,63.6
75%,38.5,98.8,200.0,202.5,192.5,198.5,217.7,145.0,145.0,117.5,...,224.5,230.0,205.0,206.4,235.0,530.7,371.1,369.5,350.0,75.2
max,98.0,300.0,555.0,577.5,595.0,592.4,595.0,551.0,500.0,577.5,...,450.0,467.5,487.5,440.5,487.5,1407.5,795.2,793.3,756.9,167.5


In [5]:
# Confirm datatypes of the features
df.dtypes

Name                 object
Sex                  object
Event                object
Equipment            object
Age                 float64
AgeClass             object
BirthYearClass       object
Division             object
BodyweightKg        float64
WeightClassKg        object
Squat1Kg            float64
Squat2Kg            float64
Squat3Kg            float64
Squat4Kg            float64
Best3SquatKg        float64
Bench1Kg            float64
Bench2Kg            float64
Bench3Kg            float64
Bench4Kg            float64
Best3BenchKg        float64
Deadlift1Kg         float64
Deadlift2Kg         float64
Deadlift3Kg         float64
Deadlift4Kg         float64
Best3DeadliftKg     float64
TotalKg             float64
Place                object
Dots                float64
Wilks               float64
Glossbrenner        float64
Goodlift            float64
Tested               object
Country              object
State                object
Federation           object
ParentFederation    

In [6]:
# Update Date from object to datetime
df['Date'] = pd.to_datetime(df['Date'])
df.dtypes

Name                        object
Sex                         object
Event                       object
Equipment                   object
Age                        float64
AgeClass                    object
BirthYearClass              object
Division                    object
BodyweightKg               float64
WeightClassKg               object
Squat1Kg                   float64
Squat2Kg                   float64
Squat3Kg                   float64
Squat4Kg                   float64
Best3SquatKg               float64
Bench1Kg                   float64
Bench2Kg                   float64
Bench3Kg                   float64
Bench4Kg                   float64
Best3BenchKg               float64
Deadlift1Kg                float64
Deadlift2Kg                float64
Deadlift3Kg                float64
Deadlift4Kg                float64
Best3DeadliftKg            float64
TotalKg                    float64
Place                       object
Dots                       float64
Wilks               

Here is a description of each feature in the provided dataset. Additional details for each feature can be found [here](https://openpowerlifting.gitlab.io/opl-csv/bulk-csv-docs.html). 

*   **Name**: Name of the lifter
*   **Sex**: Sex category in which the lifter competed (M, F, or Mx)
*   **Event**: The type of competition that the lifter entered (SBD, BD, SD, SB, S, B, D)
*   **Equipment**: Equipment category under which the lifts were performed (Raw, Wraps, Single-ply, Multi-ply, Unlimited, Straps)
*   **Age**: Age of the lifter on the start date of the meet
*   **AgeClass**: Age class in which the filter falls, based on exact age of the lifter on the day of the competition
*   **BirthYearClass**: Birth year class in which the filter falls
*   **Division**: Division of competition
*   **BodyweightKg**: Recorded bodyweight of the lifter at the time of competition
*   **WeightClassKg**: Weight class in which the lifter competed
*   **Squat1Kg, Bench1Kg, Deadlift1Kg**: First attempts for each of squat, bench, and deadlift, respectively
*   **Squat2Kg, Bench2Kg, Deadlift2Kg**: Second attempts for each of squat, bench, and deadlift, respectively
*   **Squat3Kg, Bench3Kg, Deadlift3Kg**: Third attempts for each of squat, bench, and deadlift, respectively
*   **Squat4Kg, Bench4Kg, Deadlift4Kg**: Fourth attempts for each of squat, bench, and deadlift, respectively
*   **Best3SquatKg, Best3BenchKg, Best3DeadliftKg**: Maximum of the first three successful attempts for the lift
*   **TotalKg**: Sum of Best3SquatKg, Best3BenchKg, and Best3DeadliftKg, if all three lifts were a success
*   **Place**: Recorded place of the lifter in the given division at the end of the meet
*   **Dots**: Dots points (formula used to assess strength of lifter)
*   **Wilks**: Wilks points (formula used to assess strength of lifter)
*   **Glossbrenner**: Glossbrenner points (formula used to assess strength of lifter)
*   **Goodlift**: IPF GL points (formula used to assess strength of lifter)
*   **Tested**: Yes if the lifter entered a drug-tested category, empty otherwise
*   **Country**: Home country of the lifter
*   **State**: Home state/province/oblast/division/etc of the lifter
*   **Federation**: The federation that hosted the meet
*   **ParentFederation**: Topmost federation that sanctioned the meet, usually the international body
*   **Date**: Start date of the meet
*   **MeetCountry**: Country in which the meet was held
*   **MeetState**: State, province, or region in which the meet was held
*   **MeetName**: Name of the meet

## Data cleaning

### Missing data


We will go through the different features to handle missing data.

In [7]:
# See the prevalence of missing data within our current features
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

display(missing_data)

,Total,Percent
Squat4Kg,2747930,0.997346
Bench4Kg,2736202,0.993089
Deadlift4Kg,2735844,0.992959
State,2245365,0.814943
Squat3Kg,2072871,0.752337
Squat2Kg,2052656,0.745000
Squat1Kg,2044693,0.742110
Deadlift3Kg,1995627,0.724302
Deadlift2Kg,1960029,0.711382
Deadlift1Kg,1944635,0.705794


It is expected that there will be missing data for several features, such as the bench, squat, and deadlift features (some competitions do not include all three lifts), as well as the state and country features. 

One feature that seems fairly high in missing data is age. We will try to clean and impute data for this feature. This feature is important for several analyses, including progression over time by lifters, maximum lifts by age, etc. 

We will first change age data that lists the age as less than 14 to null. According to the [2022 technical rulebook](https://www.powerlifting.sport/rules/codes/info/technical-rules) of the International Powerlifting Federation, the largest powerlifting federation in the world, the minimum age is 14. Thus, any ages below that threshold can be considered incorrect.

In [8]:
# Change any ages that are considered incorrect to null
df.loc[df.Age < 14, ['Age']] = None

# Visualize the missing rows for age
null_age_df = df[df['Age'].isnull()]
display(null_age_df.head(5))

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
76,Tatyana Labuzova,F,D,Raw,NaN,5-12,NaN,Y,64.7,65,...,Yes,Russia,NaN,GFP,NaN,2019-10-05,Russia,ORL,Oryol,Fortior est ferrum Tournament
266,Rosimeire Santos Souza,F,B,Raw,NaN,45-49,NaN,F 45-49,56.0,56,...,NaN,NaN,NaN,CONBRAP,GPC,2017-05-13,Brazil,NaN,Anhembi,Campeonato Paulista de Supino e Lev.Terra
267,João Vitor Rodrigues Claro,M,B,Raw,NaN,13-15,NaN,MT 13-15,56.0,56,...,NaN,NaN,NaN,CONBRAP,GPC,2017-05-13,Brazil,NaN,Anhembi,Campeonato Paulista de Supino e Lev.Terra
269,Eduardo Haggar da Silva,M,B,Raw,NaN,13-15,NaN,MT 13-15,60.0,60,...,NaN,NaN,NaN,CONBRAP,GPC,2017-05-13,Brazil,NaN,Anhembi,Campeonato Paulista de Supino e Lev.Terra
270,Murieu Moreira Moreno,M,B,Raw,NaN,13-15,NaN,MT 13-15,75.0,75,...,NaN,NaN,NaN,CONBRAP,GPC,2017-05-13,Brazil,NaN,Anhembi,Campeonato Paulista de Supino e Lev.Terra


We will attempt to impute missing data entries for age in two methods:


1.   Use a lifter's age from a different competition entry to calculate their age for other competitions (more accurate)
2.   Use the average of their AgeClass category or their BirthYearClass category (less accurate)



In [9]:
# Use a lifter's age from a different competition entry to calculate their age for other competitions 
lifters_missing_age = df[df['Age'].isnull()][['Name', 'Date']]
lifters_missing_age = lifters_missing_age.rename(columns={'Date':'Date_missing'})
lifters_missing_age = lifters_missing_age.drop_duplicates(subset=['Name', 'Date_missing'])

# Sort original dataset to find a row with an existing Age and Date to calculate birthday
df = df.sort_values(['Age', 'Date'])
lifters_missing_age = pd.merge(lifters_missing_age, df[['Name', 'Age', 'Date']].drop_duplicates(subset=['Name']), on='Name', how='inner')
lifters_missing_age = lifters_missing_age.rename(columns={'Age':'Age_existing', 'Date':'Date_existing'})
lifters_missing_age = lifters_missing_age[(lifters_missing_age['Age_existing'].notnull()) & (lifters_missing_age['Date_existing'].notnull())]
lifters_missing_age['Age_missing'] = lifters_missing_age['Date_missing'] - (lifters_missing_age['Date_existing'] - lifters_missing_age['Age_existing'].apply(lambda x: timedelta(seconds=x*(60*60*24*365))))
lifters_missing_age['Age_missing'] = lifters_missing_age['Age_missing'].apply(lambda x: round((x.total_seconds() / 60 / 60 / 24 / 365.25)*2)/2) # We round to the nearest 0.5 increment as the original dataset did this

display(lifters_missing_age)

,Name,Date_missing,Age_existing,Date_existing,Age_missing
28,Serafim Rocha,2017-05-13,49.5,1998-10-06,68.0
29,Serafim Rocha,2018-04-14,49.5,1998-10-06,69.0
30,Serafim Rocha,2017-07-16,49.5,1998-10-06,68.0
31,Serafim Rocha,2018-03-11,49.5,1998-10-06,69.0
32,Serafim Rocha,2018-10-21,49.5,1998-10-06,69.5
...,...,...,...,...,...
1005961,Dmitriy Dvornikov,2004-08-09,14.5,2018-05-12,0.5
1005968,Sergey Sosnovskiy,2004-08-09,17.5,2001-06-01,20.5
1006111,Erik Setnický,2009-07-01,15.5,2011-10-22,13.0
1006112,Johan Smith,2005-09-07,14.0,2006-09-05,13.0


In [26]:
# Create a copy of the original dataframe 
cleaned_df = df.copy(deep=True)

# Merge with the original dataframe to add in the estimated ages
#display(cleaned_df[cleaned_df['Name'] == 'Serafim Rocha'][['Name', 'Date', 'Age']])
cleaned_df = pd.merge(cleaned_df, lifters_missing_age[['Name', 'Date_missing', 'Age_missing']], how='left', left_on=['Name', 'Date'], right_on=['Name', 'Date_missing'])
cleaned_df.loc[cleaned_df.Age.isnull() & cleaned_df.Date_missing.notnull() & cleaned_df.Age_missing.notnull(), ['Age']] = cleaned_df['Age_missing']
# display(cleaned_df[cleaned_df['Name'] == 'Serafim Rocha'][['Name', 'Date', 'Age']])

,Name,Date,Age
2743188,Serafim Rocha,1998-10-06,49.5
2743381,Serafim Rocha,1999-10-19,50.5
2714475,Serafim Rocha,2000-10-17,51.5
2749345,Serafim Rocha,2002-10-08,53.5
2144164,Serafim Rocha,2005-10-14,56.5
...,...,...,...
567,Serafim Rocha,2018-04-14,NaN
1769,Serafim Rocha,2018-07-22,NaN
1621,Serafim Rocha,2018-10-21,NaN
2139925,Serafim Rocha,2019-10-05,NaN


We can see that there are some estimated ages that are less than 0. This suggests that either the entries with missing ages had incorrect dates or that the entries with existing ages and dates are incorrect. We will assume the former and as the 3856 rows is only 0.1% of the total dataset of 2755243 rows, we will simply remove these rows from the original dataset.

In [10]:
# See the 3856 rows with negative estimated ages
display(lifters_missing_age[lifters_missing_age['Age_missing'] < 0])


# Remove the rows with missing ages that are assumed to have incorrect dates
#ASDF 

,Name,Date_missing,Age_existing,Date_existing,Age_missing
1342,Sergey Ivanov,1993-09-26,15.5,2012-12-13,-3.5
2304,Jose Ramos,1986-04-05,24.0,2021-05-01,-11.0
3475,Terry Pierson,1998-01-31,20.5,2020-07-08,-2.0
3477,Terry Pierson,1997-10-25,20.5,2020-07-08,-2.0
3483,Terry Pierson,1998-02-28,20.5,2020-07-08,-2.0
...,...,...,...,...,...
677499,Richard Hanson,1981-04-29,26.0,2015-06-13,-8.0
677604,Einar Guðnason,1973-05-12,19.0,2010-11-03,-18.5
793290,Oscar Hernandez,1991-10-11,17.5,2018-11-18,-9.5
1003814,Shane Newman,1995-11-05,17.0,2013-04-20,-0.5


In [11]:
 # Use the average of their AgeClass category or their BirthYearClass category